In [1]:
# Преоразовать существующий код для распознавания последовательности рукописных символов в текст.
# Написать генератор произвольной последовательности рукописных символов.

# ГЕНЕРАЦИЯ НАБОРА ДАННЫХ
# Сгенерировать набор текстовых строк из допустимых символов.
# Каждому набору поставить в соотвествие массив изоражений соответствующих символов.
# Добавить пробелы для разделения слов, символы пробелов не использовать во время классификации.

# КЛАССИФИКАЦИЯ (k ближайших соседей) (пример AIM practice 2)
# Часть символов оставить для обучения классификатора . Остальные  классифицировать
# по аналогии с примером mnist с прошлого занятия. Т.е. преобразовывать изображение в вектор.
# Выбрать оптимальный классификатор в зависимости от k- количество соседей.
# Метрика для измерения точности классификации, и выбора оптимального классификатора относительно параметра k - accuracy.

# РАСПОЗНАВАНИЕ
# заменить inf_net на последовательное применение следующих шагов:
# 1. генерация изображения с последовательностью символов
# 2. Выделение объектов оставляете как есть.
# 3. Примернить k-nn для решения задачи классификации.
# 4. Вывести результат

In [ ]:
import os
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier

class HandwrittenSymbolsGenerator:
    def __init__(self, data_path):
        self.data_path = data_path
        self.symbols_data = {}
        self._load_symbols()

    def _load_symbols(self):
        folders = [f for f in os.listdir(self.data_path)
                   if os.path.isdir(os.path.join(self.data_path, f))]

        total_images = 0
        for folder in folders:
            folder_path = os.path.join(self.data_path, folder)
            images = []

            for filename in os.listdir(folder_path):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                    file_path = os.path.join(folder_path, filename)
                    images.append(file_path)

            self.symbols_data[folder] = images
            total_images += len(images)

    def generate_sequence(self, length=10):
        symbols = list(self.symbols_data.keys())
        sequence_images = []
        labels = []

        print(f"Генерация последовательности из {length} символов...")

        for i in range(length):
            symbol = random.choice(symbols)
            image_path = random.choice(self.symbols_data[symbol])
            image = cv2.imread(image_path)
            if image is not None:
                sequence_images.append(image)
                labels.append(symbol)

        sequence_image = self.create_sequence_image(sequence_images)

        print(f"Сгенерировано: {' '.join(labels)}")

        return {
            'sequence_images': sequence_images,
            'sequence_image': sequence_image,
            'labels': labels,
            'sequence_text': ''.join(labels),
            'length': length
        }

    def generate_multiple_sequences(self, num_sequences=5, length=10):
        sequences = []
        for i in range(num_sequences):
            print(f"\n--- Последовательность {i + 1} ---")
            seq_data = self.generate_sequence(length)
            sequences.append(seq_data)
        return sequences

    def create_sequence_image(self, images, spacing=20):
        processed_images = []
        for img in images:
            if len(img.shape) == 3:
                gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                processed_images.append(gray_img)
            else:
                processed_images.append(img)

        total_width = sum(img.shape[1] for img in processed_images) + spacing * (len(processed_images) - 1)
        max_height = max(img.shape[0] for img in processed_images)

        sequence_img = np.ones((max_height, total_width), dtype=np.uint8) * 255

        x_offset = 0
        for img in processed_images:
            h, w = img.shape[:2]
            y_offset = (max_height - h) // 2
            sequence_img[y_offset:y_offset + h, x_offset:x_offset + w] = img
            x_offset += w + spacing

        return sequence_img

    def display_sequence(self, sequence_data, title="Сгенерированная последовательность"):
        plt.figure(figsize=(12, 3))
        plt.imshow(sequence_data['sequence_image'], cmap='gray')
        plt.title(f"{title}: {sequence_data['sequence_text']}")
        plt.show()


DATA_PATH = "./data"
generator = HandwrittenSymbolsGenerator(DATA_PATH)

class SequenceSegmenter:
    def segment_sequence(self, sequence_image, original_labels):
        non_space_labels = [label for label in original_labels if label != ' ']
        print(f"Символы без пробелов: {non_space_labels}")

        img_gray = sequence_image
        kernel = np.ones((3, 3), np.uint8)
        img_gray = cv2.erode(img_gray, kernel, iterations=1)
        img_gray = cv2.bitwise_not(img_gray)

        base_size = img_gray.shape[0], img_gray.shape[1]
        base = np.zeros(base_size, dtype=np.uint8)
        base[0:img_gray.shape[0], 0:img_gray.shape[1]] = img_gray
        base = cv2.cvtColor(base, cv2.COLOR_GRAY2RGB)

        plt.figure(figsize=(10, 3))
        plt.imshow(base)
        plt.title(f'Последовательность: {" ".join(original_labels)}')
        plt.show()

        ret, thresh = cv2.threshold(base, 127, 255, 0)
        thresh = np.uint8(thresh)
        contours, h = cv2.findContours(thresh[:, :, 0], cv2.RETR_EXTERNAL, 2)

        contours_sorted = sorted(contours, key=lambda cnt: cv2.boundingRect(cnt)[0])
        crops = []

        img_with_boxes = base.copy()

        for i in range(len(contours_sorted)):
            cnt = contours_sorted[i]
            x, y, w, h = cv2.boundingRect(cnt)
            while (w > 45):
                w -= 1
            if w * h > 50:
                cv2.rectangle(img_with_boxes, (x, y), (x + w, y + h), (0, 0, 255), 1)
                crops.append([x, y, w, h])

        plt.figure(figsize=(10, 3))
        plt.imshow(img_with_boxes)
        plt.title('Обнаруженные контуры символов')
        plt.show()

        sequence_symbols = []
        for i in range(len(crops)):
            x, y, w, h = crops[i]

            img_crop = sequence_image[y:y + h, x:x + w]

            if i < len(non_space_labels):
                symbol_label = non_space_labels[i]
            else:
                symbol_label = "?"

            plt.figure(figsize=(3, 2))
            plt.imshow(img_crop, cmap="gray")
            plt.title(f'Символ {i + 1}: "{symbol_label}"')
            plt.axis('off')
            plt.show()

            sequence_symbols.append((symbol_label, img_crop))
            print(f"Символ {i + 1}: '{symbol_label}' ")

        return sequence_symbols


all_sequence_data = []
all_segmented_symbols = []

for i in range(2):
    print(f"Генерация #{i+1}")

    sequence_data = generator.generate_sequence(length=10)
    sequence_image = sequence_data['sequence_image']
    labels = sequence_data['labels']

    segmenter = SequenceSegmenter()
    segmented_symbols = segmenter.segment_sequence(sequence_image, labels)

    all_sequence_data.append({
        'sequence_image': sequence_image,
        'labels': labels,
        'segmented_symbols': segmented_symbols
    })
    all_segmented_symbols.append(segmented_symbols)

    print(f"Итог: сегментировано {len(segmented_symbols)} символов")


for i, data in enumerate(all_sequence_data, 1):
    print(f"\nГенерация {i}:")
    print(f"  Изображение: {data['sequence_image'].shape}")
    print(f"  Метки: {len(data['labels'])}")
    print(f"  Сегментировано символов: {len(data['segmented_symbols'])}")